In [ ]:
import ee
import geemap as gee
import geemap.eefolium as geemap
import json
import os
ee.Initialize()

In [ ]:
attributes = {'min': 20, 'max': 40,'palette': ["#000080","#0000D9","#4000FF","#8000FF","#0080FF","#00FFFF",
    "#00FF80","#80FF00","#DAFF00","#FFFF00","#FFF500","#FFDA00",
    "#FFB000","#FFA400","#FF4F00","#FF2500","#FF0A00","#FF00FF",]}
sl_geom_boundary = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', 'Sri Lanka')).geometry()
x,y = sl_geom_boundary.centroid().getInfo()['coordinates'][1],sl_geom_boundary.centroid().getInfo()['coordinates'][0]
lst_dataset = ee.ImageCollection('MODIS/061/MOD11A1')
datasets = {}

for year in range(2015,2022):
    datasets[year] = lst_dataset.filterBounds(sl_geom_boundary).filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31'))

In [ ]:
Map = geemap.Map(center=(x, y), zoom=8, lite_mode=False)

for year, data in datasets.items():
    print(f"[+] processing... {year}")
    dataset = data.select('LST_Day_1km').map(lambda img: img.multiply(0.02).subtract(273.15).copyProperties(img, ['system:time_start'])).mean().clip(sl_geom_boundary)
    Map.addLayer(dataset, attributes, 'LST ' + str(year))
    # geemap.ee_export_image(dataset, filename="D:\\" + str(year) + ".tif", scale=550, region=sl_geom_boundary, file_per_band=True)
    geemap.ee_export_image_to_drive(dataset, fileNamePrefix="sl_lst_" + str(year), region=sl_geom_boundary, folder='export', scale=550)
Map